In [2]:
import os
from dotenv import load_dotenv

# Dynamically get the path to the .env file for Jupyter/IPython
current_directory = os.getcwd()
env_path = os.path.join(current_directory, 'API_KEY.env')

# Load the .env file
load_dotenv(dotenv_path=env_path)

# Access API keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
NINJA_API_KEY = os.getenv("NINJA_API_KEY")
ALPHA_VANTAGE_API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")
FINNHUB_API_KEY = os.getenv("FINNHUB_API_KEY")

In [3]:
#### from alpha_vantage.timeseries import TimeSeries
import pandas as pd

# Your Alpha Vantage API Key
API_KEY = 

def fetch_alpha_vantage_data(ticker):
    """
    Fetch daily stock data from Alpha Vantage.
    """
    try:
        ts = TimeSeries(key=API_KEY, output_format='pandas')
        data, _ = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
        data = data.loc['2019-01-01':'2023-12-31']  # Filter for the required date range
        data = data.rename(columns={
            '5. adjusted close': 'Adj Close',
            '6. volume': 'Volume'
        })
        return data[['Adj Close', 'Volume']]
    except Exception as e:
        print(f"Error fetching data for {ticker} from Alpha Vantage: {e}")
        return None

# Example for fetching data for AAPL
data = fetch_alpha_vantage_data("AAPL")
print(data.head())


             Adj Close      Volume
date                              
2023-12-29  191.591904  42672148.0
2023-12-28  192.636788  34049898.0
2023-12-27  192.208883  47899806.0
2023-12-26  192.109371  28919310.0
2023-12-22  192.656691  37149570.0


/var/folders/ft/435zx8653tx1h7p86f714_hh0000gn/T/ipykernel_33433/415040475.py:14: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  data = data.loc['2019-01-01':'2023-12-31']  # Filter for the required date range


In [5]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import time

# Define your Alpha Vantage API Key
API_KEY = "LMSTBBNIERHKO5E6"

# Define your GICS company tickers
tickers = [
    "AAPL", "MSFT",  # Information Technology
    "JNJ", "PFE",    # Healthcare
    "JPM", "BAC",    # Financials
    "AMZN", "TSLA",  # Consumer Discretionary
    "PG", "KO",      # Consumer Staples
    "XOM", "CVX",    # Energy
    "BA", "CAT",     # Industrials
    "LIN", "DOW",    # Materials
    "AMT", "SPG",    # Real Estate
    "NEE", "DUK",    # Utilities
    "GOOGL", "META"  # Communication Services
]

# Define the market-wide performance ticker (e.g., S&P 500 ETF)
market_ticker = "SPY"

# Initialize the TimeSeries API
ts = TimeSeries(key=API_KEY, output_format="pandas")

# Function to fetch and process data
def fetch_data(ticker):
    """
    Fetch stock data from Alpha Vantage.
    """
    try:
        print(f"Fetching data for {ticker}...")
        data, _ = ts.get_daily_adjusted(symbol=ticker, outputsize="full")
        data = data.rename(columns={
            "5. adjusted close": "Adj Close",
            "6. volume": "Volume"
        })
        data = data[["Adj Close", "Volume"]]
        data.index = pd.to_datetime(data.index)
        data = data.loc["2019-01-01":"2023-12-31"]  # Filter by date range
        return data
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

# Function to calculate quarterly metrics
def calculate_quarterly_metrics(data):
    """
    Calculate quarterly metrics: start price, end price, average volume, and growth.
    """
    try:
        data['Quarter'] = data.index.to_period("Q")  # Group by quarter
        grouped = data.groupby('Quarter').agg({
            'Adj Close': ['first', 'last'],  # Start and end prices
            'Volume': 'mean',  # Average volume
        }).reset_index()
        grouped.columns = ['Quarter', 'Start_Price', 'End_Price', 'Average_Volume']
        grouped['Quarterly_Growth'] = (grouped['End_Price'] - grouped['Start_Price']) / grouped['Start_Price']
        return grouped
    except Exception as e:
        print(f"Error calculating metrics: {e}")
        return pd.DataFrame()  # Return empty DataFrame on error

# Main function to process GICS data
def process_gics_data(tickers, market_ticker):
    """
    Process data for your specific GICS tickers and the market index.
    """
    all_data = []

    # Process each ticker
    for ticker in tickers + [market_ticker]:
        data = fetch_data(ticker)
        if data is not None:
            metrics = calculate_quarterly_metrics(data)
            if not metrics.empty:
                metrics['Ticker'] = ticker
                metrics['Type'] = 'Market' if ticker == market_ticker else 'Company'
                all_data.append(metrics)
        # To prevent hitting API rate limits
        time.sleep(12)  # Alpha Vantage allows 5 API calls per minute

    # Combine all data into a single DataFrame
    if all_data:
        combined_data = pd.concat(all_data, ignore_index=True)
        return combined_data
    else:
        print("No data was processed.")
        return pd.DataFrame()

# Execute the process
gics_data = process_gics_data(tickers, market_ticker)

# Save to CSV if data exists
if not gics_data.empty:
    gics_data.to_csv("gics_stock_data_alpha_vantage.csv", index=False)
    print("Data saved to gics_stock_data_alpha_vantage.csv")
else:
    print("No data to save.")


Fetching data for AAPL...


/var/folders/ft/435zx8653tx1h7p86f714_hh0000gn/T/ipykernel_33433/3714842178.py:43: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  data = data.loc["2019-01-01":"2023-12-31"]  # Filter by date range


Fetching data for MSFT...


/var/folders/ft/435zx8653tx1h7p86f714_hh0000gn/T/ipykernel_33433/3714842178.py:43: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  data = data.loc["2019-01-01":"2023-12-31"]  # Filter by date range


Fetching data for JNJ...


...

Fetching data for META...


/var/folders/ft/435zx8653tx1h7p86f714_hh0000gn/T/ipykernel_33433/3714842178.py:43: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  data = data.loc["2019-01-01":"2023-12-31"]  # Filter by date range


Fetching data for SPY...


/var/folders/ft/435zx8653tx1h7p86f714_hh0000gn/T/ipykernel_33433/3714842178.py:43: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  data = data.loc["2019-01-01":"2023-12-31"]  # Filter by date range


Data saved to gics_stock_data_alpha_vantage.csv
